---
# 4. Simulation
---

In my simulation study I want to analyze how the variance of the PCR coefficients differ if the true VCV matrix is not known by applying the model introduced in section three. Since the covariances of some model variables are not really trackable and to have a realistic set-up, I decided to simulate a true population from which I will subsequently draw different samples for my analysis to examine the behavior of the parameter variance. 

## 4.1 Parameterization of the Model

To ease the readability I decided to create a table that includes the full name, mathematical abbreviation of section three, the chosen parameterization and a short reasoning why I chose this specific value. 

**Table X.X - Parameterization**

|Dependency| Name | Abbreviation   | Value | Reasoning | Source
|------|------|------|------|------|------|
|Ability||||||
||Variance of Ability | $\sigma^2_a$ | 1 | Since ability is not measurable and an artificial variable and an increase of the variance has the same effect as increasing $|\gamma_{ability}|$. Hence, I decided to choose the standard normal distribution for my parametrization.| -|
|General|||||
||   Population Size | N | 819,648 | There were 819.648 males aged 33 in 1991 in the UK. To have a realistic set up I decided to use the same number for my simulated population. This allows me to examine the convergence of the eigenvectors and the corresponding estimators from small samples up to very large samples (around 40000) without having the problem that the samples are very likely to be equal.| [Office for National Statistics](https://www.ons.gov.uk/file?uri=/peoplepopulationandcommunity/populationandmigration/populationestimates/datasets/populationestimatesforukenglandandwalesscotlandandnorthernireland/mid2015/ukandregionalpopulationestimates18382015.zip)|
|Years of Schooling||||||
||Maximum Years of Schooling| m | 30 | I decided 30 to allow for education between 4 and 34. | - |
||Mean/Variance of Years of Schooling | $(\mu_s, \sigma_s^2)$ | (13.342, 21.215)  | In the summary Statistics of Blundell et al. (2005) the mean/standard deviation of the years of schooling of an individual are not given. Hence, I decided to use the same moments as for parent's education.  | Blundell et al. (2005)| 
||Minimum Years of Schooling| - | 12 | I decided to use the difference between the school enrollment age and the school leaving age. The latter is 16 in Great Britain. | [British Government](https://www.gov.uk/know-when-you-can-leave-school)|
||Persistence of Parent's Education on Children's Education | $q$ | 0.85 | Using the given moments of parent's education and schooling, the coefficient that sets the lower bound for $q$ in equation *3.13* is equal to $\frac{21.215 - 13.342^2/3}{13.342 - 13.342^2/3} \approx 0.8288$. Hence, I decided to use 0.85. The choice of the coefficient does not change the results qualitatively. | -|
|Number of Siblings||||||
||Decrease in Expected Value of Number of Siblings for each Unit of Parent's Education| $p$ | 0.1 |  In Cygan-Rehm and Maeder (2013) they report this value to be 0.1. The model that is used in the paper is a linear model. However, the quantity has the same interpretation and thus I use the same number. | Cygan-Rehm and Maeder (2013)|
||Mean/Variance of Number of Siblings | $(\mu_n, \sigma_n^2)$ | (1.692, 2.89)  | From the summary Statistics of Blundell et al. (2005) | Blundell et al. (2005) |
|Parent's Education||||||
|| Mean/Variance of Years Education of Parent's | $(\mu_e, \sigma_e^2)$ | (13.342, 21.215)  | In the summary Statistics of Blundell et al. (2005) the mean/standard deviation of father's and mother's education are given. Both are given as the years of education after 6th grade. Hence I transform this by adding six to the mean. The standard deviation is not affected by this transformation. Since the moments do not meaningfully differ for fathers and mothers, I decided to only use the mother's education.  | Blundell et al. (2005) | 
|Test Scores||||||
||Influence of Ability and Parent's Education on the Latent Test Scores| $(\gamma_{ability},\gamma_{parent})$ | (2,2) | I chose $\gamma_{ability}$ and $\gamma_{parent}$ to be equal. The magnitude turned out to yield reasonable results. | - |
||Proportion of Individuals that Scored a (5, 4, 3, 2, 1) in 7th Grade Math | $Gr_{7,m}$ | (0.141, 0.158, 0.185, 0.190, 0.212)| I chose to use the reported quintiles for each test as approximation of the probability mass.| Blundell et al. (2005)|
||Proportion of Individuals that Scored a (5, 4, 3, 2, 1) in 11th Grade Math | $Gr_{11,m}$ | (0.122,0.152, 0.157, 0.179, 0.199)| I chose to use the reported quintiles for each test as approximation of the probability mass.| Blundell et al. (2005)|
||Proportion of Individuals that Scored a (5, 4, 3, 2, 1) in 7th Grade Reading | $Gr_{7,r}$ | (0.166, 0.179, 0.188, 0.187, 0.165)| I chose to use the reported quintiles for each test as approximation of the probability mass.| Blundell et al. (2005)|
||Proportion of Individuals that Scored a (5, 4, 3, 2, 1) in 11th Grade Reading | $Gr_{11,r}$ | (0.132, 0.163, 0.163, 0.176, 0.176)| I chose to use the reported quintiles for each test as approximation of the probability mass.| Blundell et al. (2005)|
|| Variance of Normally Distributed Errors for Test Scores| $\sigma^2$| 1 | Higher variance of the error decreases the correlation of ability and parent's education with the test outcomes. This parameter must be seen relatively to $\gamma_{ability}$ and $\gamma_{parent}$. Hence, I chose standard normal distributed errors to allow for high correlation between the variables without being forced to use very high values of $\gamma_{ability}$ and $\gamma_{parent}$.| - | 
|Wages||||||
||Lower Bounds for the Betas in the Wage Regression (schooling, working, working_sqr/100, number of Siblings, parent's education)| $\beta^{min}$ | (0.03, 0.01, -0.06, -10, 0.01) | The values for schooling, work experience and the squared work experience are chosen such that they are close to the values in Björklund and Kjellström (2002). For the number of Siblings I wanted to set a non-binding bound. To allow changes in this, I found it reasonable to also incorporate the constraint in the code and set it to a very high value. Parent's education should have a positive influence on the wages, hence I chose a small positive number. | Björklund and Kjellström (2002) |
||Mean/Variance of Logarithmic Wage| $(\mu_Y, \sigma^2_Y)$ | (2.040, 1.5) |  Blundell et al. (2005) | Blundell et al. (2005) |
||Upper Bounds for the Betas in the Wage Regression (constant, schooling, working, working_sqr/100, number of Siblings, parent's education) | $\beta^{max}$ | (0.06, 0.06, -0.03, 10, 10) | The values for schooling, work experience and the squared work experience are chosen such that they are close to the values in Björklund and Kjellström (2002). For the number of Siblings and parent's education I wanted to set a non-binding upper bound and thus choose a high value for the coefficients. To allow changes in the latter two quantities, I found it reasonable to also incorporate the constraint in the code.  | Björklund and Kjellström (2002) |
||Weighting of Squarred Deviation to Expected Value and Standard Deviation for the Logarithmic Wage | $\tau$ | 0.5 | I choose equal weights for the Expected Value and the Standard Deviation. A change in $\tau$ did only change the sresults in a very small magnitude.| - |
|Years of Working experience||||||
||Age at School Enrollment | $age\_schoolenrollment$ | 4 | The enrollment age for school of most children is four in Great Britain. |  [British Government](https://www.gov.uk/schools-admissions/school-starting-age)|
||Age of all Individuals | $age$ | 33 | I used the same value as the one reported in Blundell et al. (2005) since most of the moments are taken from the paper.| Blundell et al. (2005)|
||Probability Mass of Gap Years (0,1,2,3,4) | $F_{gap}$ |  (0.59, 0.11,0.7,0.04, 0.03) | Unfortunately I could not find data for the UK. Holmlund et al. (2008) report swedish data for gap years between schooling and university in 1991. I use this data and assume that everyone that does not go to university does not take gap any gap year.  | Holmlund et al. (2008) |